In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import random
import dask.dataframe as dd

import warnings
warnings.filterwarnings('ignore')

Load the data

In [2]:
df = pd.read_csv('csv/credit_card_fraud_sample_10K.csv')

In [3]:
df.head()

,Unnamed: 0,ssn,cc_num,first,last,gender,street,city,state,zip,...,trans_num,trans_date,trans_time,unix_time,category,amt,is_fraud,merchant,merch_lat,merch_long
0,8208382,421-07-7177,3534945553430195,Shelia,Fleming,F,153 Dennis Crossing,Weston,WV,26452,...,5b159948cb3f4e2b1a82dbe014e2f7db,2020-06-04,19:55:30,1591280730,health_fitness,16.74,0,fraud_Hirthe-Beier,38.224419,-80.100552
1,12606453,061-53-6686,4183314058259431806,Nicole,Todd,F,088 Jeffrey Prairie Apt. 394,Mountain Center,CA,92561,...,93550825285d02993ed8a30d269025c8,2019-11-10,01:16:24,1573328784,shopping_net,745.63,0,fraud_Schmidt and Sons,32.952717,-116.368019
2,28301366,038-31-5036,2261044046656441,Matthew,Mann,M,3315 Holmes Junctions,Greensboro,NC,27410,...,b6a1fc16622b0ed557f0af093ecc8512,2020-08-12,20:29:53,1597244393,home,94.13,0,"fraud_Bahringer, Bergnaum and Quitzon",35.419988,-80.107734
3,11771739,830-11-6351,374795838353160,Carolyn,Hall,F,242 Carroll Bypass,Spokane,WA,99208,...,2e944bdc8faa762005f0b0486010d307,2019-09-21,02:28:11,1569013091,shopping_net,73.43,0,"fraud_Heathcote, Yost and Kertzmann",47.441771,-117.255467
4,12450008,408-99-8998,4122635746026142330,Andre,Rice,M,8085 Burgess Junctions,Myrtle Beach,SC,29588,...,f4c5d40c54cb268beb8f501df22785d6,2019-04-21,03:00:46,1555795846,gas_transport,35.49,0,"fraud_Raynor, Reinger and Hagenes",33.911297,-78.057010


In [6]:
df.isna().sum()

Unnamed: 0    0
ssn           0
cc_num        0
first         0
last          0
gender        0
street        0
city          0
state         0
zip           0
lat           0
long          0
city_pop      0
job           0
dob           0
acct_num      0
profile       0
trans_num     0
trans_date    0
trans_time    0
unix_time     0
category      0
amt           0
is_fraud      0
merchant      0
merch_lat     0
merch_long    0
dtype: int64

Clean the Data
1. consider drop first/last name, ssn, cc_num, trans_num etc...
2. explore locational data e.g zip lat/long, city etc... TBD replace with regions??
3. reorganize date & time data:
    3.1 work day vs weekend
    3.2 TBD: split hour segment, work/sleep/free hours
4. set 4 age groups youngest to oldest (18-199), teenager (-18) young_adult (19-25), adult (26-65), retired (...)
    4.1 TBD treat different for men & women?? retirement age 66(M), 63(F)

5. categorize: category,          

6. explore: amt, dob? 


In [5]:
df.nunique()

Unnamed: 0    10000
ssn            7155
cc_num         7155
first           635
last            957
gender            2
street         7155
city           3030
state            51
zip            5348
lat            5237
long           5306
city_pop       3535
job             639
dob            6130
acct_num       7155
profile          12
trans_num     10000
trans_date      731
trans_time     9382
unix_time      9999
category         14
amt            6608
is_fraud          2
merchant        693
merch_lat      9996
merch_long    10000
dtype: int64